In [1]:
!pip install -U langchain-community pypdf faiss-cpu bitsandbytes

In [2]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [3]:
model_id = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")



tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [4]:
loaders = []
for filename in os.listdir("pdf_books/"):
    if filename.endswith(".pdf"):
        loaders.append(PyPDFLoader(os.path.join("pdf_books", filename)))

documents = []
for loader in loaders:
    documents.extend(loader.load())


In [5]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.split_documents(documents)

In [6]:
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")

/tmp/ipython-input-3780940678.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-mpnet-base-v2")


In [7]:
vectorstore = FAISS.from_documents(docs, embeddings)

In [8]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.1,
    top_p=0.95,
    repetition_penalty=1.1,
    do_sample=True
)

Device set to use cuda:0


In [31]:
llm = HuggingFacePipeline(pipeline=pipe)

prompt_template = """
You are an expert mathematical assistant. You always provide clear, step-by-step and accurate answers using only the provided context.

Use ONLY the context below to answer the question at the end.
If you don’t know the answer, just say “I don’t know” — do NOT try to make up an answer.

---------------------
Context:
{context}
---------------------

Question: {question}

Helpful, structured answer:
"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template
)

In [32]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt}
)


In [33]:
query = "what is graph "
response = qa.run(query)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [34]:
print(response)


You are an expert mathematical assistant. You always provide clear, step-by-step and accurate answers using only the provided context.

Use ONLY the context below to answer the question at the end.
If you don’t know the answer, just say “I don’t know” — do NOT try to make up an answer.

---------------------
Context:
vertices
Figure 6.4 Names of parts of a graph.
A sequence of n nonnegative numbers, dl, d2.  d, is said to be graphical if there
exists a graph having n vertices with the degrees of the vertices being dl, d2 ..... d, The
degrees of the vertices of a graph are called a degree sequence. Without loss of generality,
one can assume that d, < d2 < ... < dn. The graph in Figure 6.4 has degree sequence 0,
2,2,3,3,4.
Definition 2. A graph on n vertices having each pair of distinct vertices joined by an

(b) The Diameter of the Graph: The maximum distance between any two vertices.
For example, in a path with n vertices, the diameter is n. For such a graph, it
can take a relatively 